In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [21]:
import os

from PIL import Image

import numpy as np

import pandas as pd

from sklearn import datasets, model_selection

In [22]:
dirs = ['ants','bees']

data = []
label = []

for i, d in enumerate(dirs):
  files = os.listdir('./drive/My Drive/hymenoptera_data/train/' + d)

  for f in files:
    img = Image.open('./drive/My Drive/hymenoptera_data/train/' + d + '/' + f, 'r')

    resize_img = img.resize((128, 128))
    if resize_img.mode == "RGB":
      r,g,b = resize_img.split()
    else:
      continue
    r_resize_img = np.asarray(np.float32(r)/255.0)
    g_resize_img = np.asarray(np.float32(g)/255.0)
    b_resize_img = np.asarray(np.float32(b)/255.0)

    rgb_resize_img = np.asarray([r_resize_img,g_resize_img,b_resize_img])

    data.append(rgb_resize_img)


    label.append(i)

In [23]:
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

data.shape

(244, 3, 128, 128)

In [26]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(data, label, test_size = 0.1)

print(len(train_x))
print(len(test_x))

219
25


In [27]:
train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).long()

print(train_x.shape)
print(train_y.shape)

torch.Size([219, 3, 128, 128])
torch.Size([219])


In [28]:
train = TensorDataset(train_x, train_y)

print(train[0])

train_loader = DataLoader(train, batch_size=32, shuffle=True)

(tensor([[[0.1608, 0.1608, 0.1608,  ..., 0.0706, 0.0745, 0.0706],
         [0.1529, 0.1529, 0.1529,  ..., 0.0706, 0.0784, 0.0706],
         [0.1412, 0.1451, 0.1451,  ..., 0.0745, 0.0824, 0.0745],
         ...,
         [0.7373, 0.7922, 0.8157,  ..., 0.5843, 0.6392, 0.6510],
         [0.7451, 0.6784, 0.7059,  ..., 0.6275, 0.6549, 0.6588],
         [0.5725, 0.5529, 0.5961,  ..., 0.6471, 0.6745, 0.7020]],

        [[0.1804, 0.1725, 0.1725,  ..., 0.0824, 0.0824, 0.0824],
         [0.1686, 0.1647, 0.1647,  ..., 0.0824, 0.0863, 0.0824],
         [0.1529, 0.1529, 0.1569,  ..., 0.0902, 0.0902, 0.0863],
         ...,
         [0.7725, 0.8275, 0.8431,  ..., 0.6039, 0.6510, 0.6706],
         [0.7922, 0.6902, 0.7255,  ..., 0.6471, 0.6706, 0.6706],
         [0.5686, 0.5255, 0.5922,  ..., 0.6667, 0.6784, 0.7020]],

        [[0.5569, 0.5647, 0.5765,  ..., 0.1765, 0.1725, 0.1725],
         [0.5412, 0.5490, 0.5490,  ..., 0.1843, 0.1765, 0.1804],
         [0.5098, 0.5216, 0.5255,  ..., 0.1882, 0.1882, 0

In [52]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3,10,5)
    self.conv2 = nn.Conv2d(10,15,5)

    self.fc1 = nn.Linear(15*29*29,50)
    self.fc2 = nn.Linear(50,2)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)),2)
    x = F.max_pool2d(F.relu(self.conv2(x)),2)
    x = x.view(-1,15*29*29)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x)


model = Net()

In [53]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(),lr = 0.001)

for epoch in range(300):
  total_loss = 0

  for train_x, train_y in train_loader:
    train_x, train_y = Variable(train_x), Variable(train_y)

    optimizer.zero_grad()

    output = model(train_x)

    loss = criterion(output, train_y)

    loss.backward()

    optimizer.step()
  if (epoch + 1) % 30 == 0:
    print(epoch +1, loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


30 tensor(0.0018, grad_fn=<NllLossBackward>)
60 tensor(0.0004, grad_fn=<NllLossBackward>)
90 tensor(0.0002, grad_fn=<NllLossBackward>)
120 tensor(3.6154e-05, grad_fn=<NllLossBackward>)
150 tensor(4.6404e-05, grad_fn=<NllLossBackward>)
180 tensor(2.3258e-05, grad_fn=<NllLossBackward>)
210 tensor(1.8689e-05, grad_fn=<NllLossBackward>)
240 tensor(2.5779e-05, grad_fn=<NllLossBackward>)
270 tensor(1.7139e-05, grad_fn=<NllLossBackward>)
300 tensor(1.0932e-05, grad_fn=<NllLossBackward>)


In [54]:
test_x = np.array(test_x, dtype='float32')
test_y = np.array(test_y, dtype='int64')

test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).long()

print(test_x.shape)
print(test_y.shape)

torch.Size([25, 3, 128, 128])
torch.Size([25])


In [55]:
test_x, test_y = Variable(test_x),Variable(test_y)

result = torch.max(model(test_x).data, 1)[1]

accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

accuracy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


0.76